# Data Wrangling with Twitter
Jessica Ertel

In [103]:
# data gathering 
import config
import os
import requests
import time
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# standard data manipulation libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

## Data Gathering
Using the tweet IDs from the WeRateDogs Twitter archive, I query the Twitter API for each tweet's JSON data and write the data to a text file.

In [104]:
# import WeRateDogs twitter archive (provided by Udacity)
archive = pd.read_csv('twitter-archive-enhanced.csv')

In [5]:
# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
consumer_key = config.CONSUMER_KEY
consumer_secret = config.CONSUMER_SECRET
access_token = config.ACCESS_TOKEN
access_secret = config.ACCESS_SECRET

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Gather IDs from WeRateDogs twitter archive
tweet_ids = archive.tweet_id.values


# Query Twitter's API for JSON data for each tweet ID
# Save each tweet's returned JSON as a new line in a .txt file
fails_dict = {}

with open('tweet_json.txt', 'w') as outfile:
    for tweet_id in tweet_ids:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print(f"Tweet ID: {tweet_id} no longer exists.") 
            fails_dict[tweet_id] = e
            pass

print(fails_dict)

Tweet ID: 888202515573088257 no longer exists.
Tweet ID: 873697596434513921 no longer exists.
Tweet ID: 872668790621863937 no longer exists.
Tweet ID: 872261713294495745 no longer exists.
Tweet ID: 869988702071779329 no longer exists.
Tweet ID: 866816280283807744 no longer exists.
Tweet ID: 861769973181624320 no longer exists.
Tweet ID: 856602993587888130 no longer exists.
Tweet ID: 851953902622658560 no longer exists.
Tweet ID: 845459076796616705 no longer exists.
Tweet ID: 844704788403113984 no longer exists.
Tweet ID: 842892208864923648 no longer exists.
Tweet ID: 837366284874571778 no longer exists.
Tweet ID: 837012587749474308 no longer exists.
Tweet ID: 829374341691346946 no longer exists.
Tweet ID: 827228250799742977 no longer exists.
Tweet ID: 812747805718642688 no longer exists.
Tweet ID: 802247111496568832 no longer exists.
Tweet ID: 779123168116150273 no longer exists.
Tweet ID: 775096608509886464 no longer exists.
Tweet ID: 771004394259247104 no longer exists.
Tweet ID: 770

In [16]:
# Load in json data line by line and turn into pandas df
data = []
with open('tweet-json.txt') as json_file:
    for row in json_file:
        json_data = json.loads(row)
        data.append({"tweet_id":json_data["id"],
                   "favorites":json_data["favorite_count"],
                   "retweets":json_data["retweet_count"],
                   "timestamp":json_data["created_at"]})

tweets = pd.DataFrame(data, columns=['tweet_id',
                                     'favorites',
                                     'retweets',
                                     'timestamp'])

In [17]:
tweets.head()

,tweet_id,favorites,retweets,timestamp
0,892420643555336193,39467,8853,Tue Aug 01 16:23:56 +0000 2017
1,892177421306343426,33819,6514,Tue Aug 01 00:17:27 +0000 2017
2,891815181378084864,25461,4328,Mon Jul 31 00:18:03 +0000 2017
3,891689557279858688,42908,8964,Sun Jul 30 15:58:51 +0000 2017
4,891327558926688256,41048,9774,Sat Jul 29 16:00:24 +0000 2017


In [18]:
# Save to csv
tweets.to_csv('twitter_scrape.csv', index=False)

In [105]:
# download image predictions programmatically (provided by Udacity)
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
preds = requests.get(url,auth=('user', 'pass'))

with open('image_predictions.tsv', 'wb') as file:
    file.write(preds.content)

image_preds = pd.read_csv('image_predictions.tsv', sep='\t')
image_preds.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


## Data Assessing
After gathering the data, I assess them visually and programmatically for quality and tidiness issues. I detect and document eight quality issues and two tidiness issues.

### Assessment
Tidiness  
Each variable forms a column and each observation forms a row. Ex: splitting info in one column into two. How many tables do you need?  
1. Combine the tables (archive, tweets, image_preds) and remove duplicated columns.
2. Address missing data.

Quality
1. Check data types
2. Check timestamp is the same
3. Each row is identified as doggo, floofer, pupper or puppo
4. If all three predictions are not breeds of dogs, then drop the row (likely not a dog).
5. Drop all ratings that do not have image predictions
6. Update names identified as 'just'
7. Update rating denominators (some are incorrectly pulled from notes)

## Data Cleaning: Tidiness

In [106]:
tweets = pd.read_csv('twitter_scrape.csv')
tweets.shape

(2354, 4)

In [107]:
image_preds.shape

(2075, 12)

In [108]:
archive.shape

(2356, 17)

### Merge Tables

In [109]:
# merge image preds, tweets and archive datasets
full = pd.merge(pd.merge(image_preds, tweets, on='tweet_id'), archive, on='tweet_id')
full.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,...,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,...,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,...,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,...,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,...,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,...,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None


In [110]:
full.shape

(2073, 31)

In [111]:
full.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'favorites', 'retweets',
       'timestamp_x', 'in_reply_to_status_id', 'in_reply_to_user_id',
       'timestamp_y', 'source', 'text', 'retweeted_status_id',
       'retweeted_status_user_id', 'retweeted_status_timestamp',
       'expanded_urls', 'rating_numerator', 'rating_denominator', 'name',
       'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

### Address Missing Data

In [112]:
# Check percentage of null values per column
full.isnull().sum() / len(full) * 100

tweet_id                       0.000000
jpg_url                        0.000000
img_num                        0.000000
p1                             0.000000
p1_conf                        0.000000
p1_dog                         0.000000
p2                             0.000000
p2_conf                        0.000000
p2_dog                         0.000000
p3                             0.000000
p3_conf                        0.000000
p3_dog                         0.000000
favorites                      0.000000
retweets                       0.000000
timestamp_x                    0.000000
in_reply_to_status_id         98.890497
in_reply_to_user_id           98.890497
timestamp_y                    0.000000
source                         0.000000
text                           0.000000
retweeted_status_id           96.189098
retweeted_status_user_id      96.189098
retweeted_status_timestamp    96.189098
expanded_urls                  0.000000
rating_numerator               0.000000


In [113]:
# Drop columns with over 95% nulls
full.drop(columns=['in_reply_to_status_id', 
                        'in_reply_to_user_id', 
                        'retweeted_status_id', 
                        'retweeted_status_user_id',
                        'retweeted_status_timestamp'], inplace=True)

## Data Cleaning: Quality

### Combine Dog Stages into one column

In [114]:
# replace none with an empty string
full[full.columns[22:]].apply(lambda x: x.replace('None', ''), axis=1)

,doggo,floofer,pupper,puppo
0,,,,
1,,,,
2,,,,
3,,,,
4,,,,
...,...,...,...,...
2068,,,,
2069,,,,
2070,,,,
2071,,,,


In [115]:
full['stage'] = full[full.columns[22:]].apply(lambda x: ','.join(x), axis=1)

### Update Incorrect Denominators

In [85]:
full[(full['rating_denominator'] != 10)]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,...,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
20,666287406224695296,https://pbs.twimg.com/media/CT8g3BpUEAAuFjg.jpg,1,Maltese_dog,0.857531,True,toy_poodle,0.063064,True,miniature_poodle,...,"<a href=""http://twitter.com/download/iphone"" r...",This is an Albanian 3 1/2 legged Episcopalian...,https://twitter.com/dog_rates/status/666287406...,1,2,an,None,None,None,None
501,675853064436391936,https://pbs.twimg.com/media/CWEcxqWVEAAHyGH.jpg,1,Labrador_retriever,0.868367,True,golden_retriever,0.043305,True,vizsla,...,"<a href=""http://twitter.com/download/iphone"" r...",Here we have an entire platoon of puppers. Tot...,https://twitter.com/dog_rates/status/675853064...,88,80,None,None,None,None,None
560,677716515794329600,https://pbs.twimg.com/media/CWe7kw9W4AE8UJh.jpg,1,teddy,0.662908,False,crib,0.031891,False,chow,...,"<a href=""http://twitter.com/download/iphone"" r...",IT'S PUPPERGEDDON. Total of 144/120 ...I think...,https://twitter.com/dog_rates/status/677716515...,144,120,None,None,None,None,None
667,682962037429899265,https://pbs.twimg.com/media/CXpeVzQW8AApKYb.jpg,1,dingo,0.278600,False,Chihuahua,0.155207,True,loupe,...,"<a href=""http://twitter.com/download/iphone"" r...",This is Darrel. He just robbed a 7/11 and is i...,https://twitter.com/dog_rates/status/682962037...,7,11,Darrel,None,None,None,None
692,684222868335505415,https://pbs.twimg.com/media/CX7Y_ByWwAEJdUy.jpg,1,soft-coated_wheaten_terrier,0.791182,True,cocker_spaniel,0.072444,True,teddy,...,"<a href=""http://twitter.com/download/iphone"" r...",Someone help the girl is being mugged. Several...,https://twitter.com/dog_rates/status/684222868...,121,110,None,None,None,None,None
693,684225744407494656,https://pbs.twimg.com/media/CX7br3HWsAAQ9L1.jpg,2,golden_retriever,0.203249,True,Samoyed,0.067958,True,Great_Pyrenees,...,"<a href=""http://twitter.com/download/iphone"" r...","Two sneaky puppers were not initially seen, mo...",https://twitter.com/dog_rates/status/684225744...,143,130,None,None,None,None,None
865,697463031882764288,https://pbs.twimg.com/media/Ca3i7CzXIAMLhg8.jpg,1,Labrador_retriever,0.999885,True,golden_retriever,0.000098,True,pug,...,"<a href=""http://twitter.com/download/iphone"" r...",Happy Wednesday here's a bucket of pups. 44/40...,https://twitter.com/dog_rates/status/697463031...,44,40,None,None,None,None,None
941,704054845121142784,https://pbs.twimg.com/media/CcVOJEcXEAM0FHL.jpg,1,Great_Pyrenees,0.667939,True,kuvasz,0.228764,True,golden_retriever,...,"<a href=""http://twitter.com/download/iphone"" r...",Here is a whole flock of puppers. 60/50 I'll ...,https://twitter.com/dog_rates/status/704054845...,60,50,a,None,None,None,None
1007,709198395643068416,https://pbs.twimg.com/media/CdeUKpcWoAAJAWJ.jpg,1,borzoi,0.490783,True,wire-haired_fox_terrier,0.083513,True,English_setter,...,"<a href=""http://twitter.com/download/iphone"" r...","From left to right:\nCletus, Jerome, Alejandro...",https://twitter.com/dog_rates/status/709198395...,45,50,None,None,None,None,None
1025,710658690886586372,https://pbs.twimg.com/media/CdzETn4W4AAVU5N.jpg,1,soft-coated_wheaten_terrier,0.948617,True,Dandie_Dinmont,0.018664,True,cairn,...,"<a href=""http://twitter.com/download/iphone"" r...",Here's a brigade of puppers. All look very pre...,https://twitter.com/dog_rates/status/710658690...,80,80,None,None,None,None,None


### Timestamp

In [47]:
tweets_full['timestamp_x']

0       Tue Aug 01 16:23:56 +0000 2017
1       Tue Aug 01 00:17:27 +0000 2017
2       Mon Jul 31 00:18:03 +0000 2017
3       Sun Jul 30 15:58:51 +0000 2017
4       Sat Jul 29 16:00:24 +0000 2017
                     ...              
2349    Mon Nov 16 00:24:50 +0000 2015
2350    Mon Nov 16 00:04:52 +0000 2015
2351    Sun Nov 15 23:21:54 +0000 2015
2352    Sun Nov 15 23:05:30 +0000 2015
2353    Sun Nov 15 22:32:08 +0000 2015
Name: timestamp_x, Length: 2354, dtype: object

In [50]:
# Drop the day of the week in timestamp_x
# strip_day = [timestamp[4:] for timestamp in tweets_full['timestamp_x']]

# tweets_full['timestamp_x'].apply(strip_day)

In [38]:
tweets_full['timestamp_x']

0        Aug 01 16:23:56 +0000 2017
1        Aug 01 00:17:27 +0000 2017
2        Jul 31 00:18:03 +0000 2017
3        Jul 30 15:58:51 +0000 2017
4        Jul 29 16:00:24 +0000 2017
                   ...             
2349     Nov 16 00:24:50 +0000 2015
2350     Nov 16 00:04:52 +0000 2015
2351     Nov 15 23:21:54 +0000 2015
2352     Nov 15 23:05:30 +0000 2015
2353     Nov 15 22:32:08 +0000 2015
Name: timestamp_x, Length: 2354, dtype: object

In [51]:
# Change the timestamp columns to timestamp format
#tweets_full[['timestamp_x','timestamp_y']]=tweets_full[['timestamp_x','timestamp_y']].apply(pd.to_datetime, format='%m-%d-%Y %H:%M:%S.%f')

In [64]:
# Can a dog identify in multiple stages?
tweets_full[(tweets_full['doggo']=='doggo')&(tweets_full['floofer']=='floofer')]

,tweet_id,favorites,retweets,timestamp_x,timestamp_y,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
199,854010172552949760,17169,3433,Mon Apr 17 16:34:26 +0000 2017,2017-04-17 16:34:26 +0000,"At first I thought this was a shy doggo, but i...",11,10,None,doggo,floofer,None,None


In [66]:
tweets_full[(tweets_full['doggo']=='doggo')&(tweets_full['pupper']=='pupper')]

,tweet_id,favorites,retweets,timestamp_x,timestamp_y,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
459,817777686764523521,11901,3084,Sat Jan 07 16:59:28 +0000 2017,2017-01-07 16:59:28 +0000,"This is Dido. She's playing the lead role in ""...",13,10,Dido,doggo,None,pupper,None
530,808106460588765185,9701,2525,Mon Dec 12 00:29:28 +0000 2016,2016-12-12 00:29:28 +0000,Here we have Burke (pupper) and Dexter (doggo)...,12,10,None,doggo,None,pupper,None
564,802265048156610565,7039,1573,Fri Nov 25 21:37:47 +0000 2016,2016-11-25 21:37:47 +0000,"Like doggo, like pupper version 2. Both 11/10 ...",11,10,None,doggo,None,pupper,None
574,801115127852503040,8992,2429,Tue Nov 22 17:28:25 +0000 2016,2016-11-22 17:28:25 +0000,This is Bones. He's being haunted by another d...,12,10,Bones,doggo,None,pupper,None
704,785639753186217984,8735,2561,Tue Oct 11 00:34:48 +0000 2016,2016-10-11 00:34:48 +0000,This is Pinot. He's a sophisticated doggo. You...,10,10,Pinot,doggo,None,pupper,None
732,781308096455073793,7973,2961,Thu Sep 29 01:42:20 +0000 2016,2016-09-29 01:42:20 +0000,"Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",12,10,None,doggo,None,pupper,None
777,775898661951791106,0,17621,Wed Sep 14 03:27:11 +0000 2016,2016-09-14 03:27:11 +0000,"RT @dog_rates: Like father (doggo), like son (...",12,10,None,doggo,None,pupper,None
820,770093767776997377,0,3520,Mon Aug 29 03:00:36 +0000 2016,2016-08-29 03:00:36 +0000,RT @dog_rates: This is just downright precious...,12,10,just,doggo,None,pupper,None
887,759793422261743616,6620,2173,Sun Jul 31 16:50:42 +0000 2016,2016-07-31 16:50:42 +0000,"Meet Maggie &amp; Lila. Maggie is the doggo, L...",12,10,Maggie,doggo,None,pupper,None
954,751583847268179968,4849,1265,Sat Jul 09 01:08:47 +0000 2016,2016-07-09 01:08:47 +0000,Please stop sending it pictures that don't eve...,5,10,None,doggo,None,pupper,None


In [68]:
tweets_full[(tweets_full['name']=='just')]

,tweet_id,favorites,retweets,timestamp_x,timestamp_y,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
820,770093767776997377,0,3520,Mon Aug 29 03:00:36 +0000 2016,2016-08-29 03:00:36 +0000,RT @dog_rates: This is just downright precious...,12,10,just,doggo,None,pupper,None
1061,741067306818797568,10342,3520,Fri Jun 10 00:39:48 +0000 2016,2016-06-10 00:39:48 +0000,This is just downright precious af. 12/10 for ...,12,10,just,doggo,None,pupper,None
1455,695095422348574720,2888,684,Thu Feb 04 04:03:57 +0000 2016,2016-02-04 04:03:57 +0000,This is just a beautiful pupper good shit evol...,12,10,just,None,None,pupper,None
2017,672125275208069120,2578,1253,Wed Dec 02 18:48:47 +0000 2015,2015-12-02 18:48:47 +0000,This is just impressive I have nothing else to...,11,10,just,None,None,None,None


### Remove non-dog images

In [117]:
# Are there non-dog images in the dataset?
full[(full['p1_dog']==False) & (full['p2_dog']==False) & (full['p3_dog']==False)]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,...,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,stage
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,...,This is an odd dog. Hard on the outside but lo...,https://twitter.com/dog_rates/status/666051853...,2,10,an,None,None,None,None,"None,None,None,None"
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,0.033919,False,partridge,...,Not familiar with this breed. No tail (weird)....,https://twitter.com/dog_rates/status/666104133...,1,10,None,None,None,None,None,"None,None,None,None"
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,desktop_computer,0.086502,False,desk,0.085547,False,bookcase,...,Very concerned about fellow dog trapped in com...,https://twitter.com/dog_rates/status/666268910...,10,10,None,None,None,None,None,"None,None,None,None"
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,three-toed_sloth,0.914671,False,otter,0.015250,False,great_grey_owl,...,This is a funny dog. Weird toes. Won't come do...,https://twitter.com/dog_rates/status/666293911...,3,10,a,None,None,None,None,"None,None,None,None"
25,666362758909284353,https://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg,1,guinea_pig,0.996496,False,skunk,0.002402,False,hamster,...,Unique dog here. Very small. Lives in containe...,https://twitter.com/dog_rates/status/666362758...,6,10,None,None,None,None,None,"None,None,None,None"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,880935762899988482,https://pbs.twimg.com/media/DDm2Z5aXUAEDS2u.jpg,1,street_sign,0.251801,False,umbrella,0.115123,False,traffic_light,...,This is Louis. He's crossing. It's a big deal....,https://twitter.com/dog_rates/status/880935762...,13,10,Louis,None,None,None,None,"None,None,None,None"
2021,881268444196462592,https://pbs.twimg.com/media/DDrk-f9WAAI-WQv.jpg,1,tusker,0.473303,False,Indian_elephant,0.245646,False,ibex,...,Meet Elliot. He's a Canadian Forrest Pup. Unus...,https://twitter.com/dog_rates/status/881268444...,12,10,Elliot,None,None,None,None,"None,None,None,None"
2045,886680336477933568,https://pbs.twimg.com/media/DE4fEDzWAAAyHMM.jpg,1,convertible,0.738995,False,sports_car,0.139952,False,car_wheel,...,This is Derek. He's late for a dog meeting. 13...,https://twitter.com/dog_rates/status/886680336...,13,10,Derek,None,None,None,None,"None,None,None,None"
2051,887517139158093824,https://pbs.twimg.com/ext_tw_video_thumb/88751...,1,limousine,0.130432,False,tow_truck,0.029175,False,shopping_cart,...,I've yet to rate a Venezuelan Hover Wiener. Th...,https://twitter.com/dog_rates/status/887517139...,14,10,such,None,None,None,None,"None,None,None,None"


In [61]:
image_preds['img_num'].value_counts()

1    1780
2     198
3      66
4      31
Name: img_num, dtype: int64

## Analyze and Visualize
Analyze and visualize the wrangled data. I produce three (3) insights and one (1) visualization.

### Insights